In [ ]:
# 1. Install dependencies
!pip install streamlit faiss-cpu sentence-transformers langchain-community pyngrok pypdf groq langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/66

In [ ]:
!pip install streamlit \
langchain==0.1.16 langchain-core==0.3.48 langchain-community==0.3.23 \
langsmith==0.1.125 langchain-groq==0.3.2 \
faiss-cpu sentence-transformers PyPDF2 pillow \
httpx==0.27.0 openai==1.17.0 \
deep-translator

INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install langchain-community==0.3.23 and langchain==0.1.16 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested langchain-community==0.3.23
    langchain 0.1.16 depends on langchain-community<0.1 and >=0.0.32

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 31.9 MB/s eta 0:00:00


In [ ]:
!pip install deep-translator


  Using cached deep_translator-1.11.4-py3-none-any.whl.metadata (30 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import streamlit as st
import speech_recognition as sr
from deep_translator import GoogleTranslator
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from PIL import Image
from io import BytesIO
import base64

# ============ CONFIG ============
os.environ["GROQ_API_KEY"] = "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"
MODEL_NAME = "llama3-8b-8192"
PDF_PATHS = {
    "Pharmacology": "/content/drive/MyDrive/Lippincott Illustrated Reviews Pharmacology Seventh Edition.pdf",
    "Pain management": "/content/drive/MyDrive/pain_wise_a_patients_guide_to_pain_management_1nbsped_1578264081.pdf",
    "First aid": "/content/drive/MyDrive/New-Vital-First-Aid-First-Aid-Book-112019.pdf"
}
# ================================


def load_medical_docs(path):
    loader = PyPDFLoader(path)
    docs = loader.load()
    print(f"{len(docs)}")
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(docs)

def embed_documents(docs):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.from_documents(docs, embeddings)

def build_qa_system(faiss_index):
    llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"), model_name=MODEL_NAME)
    return RetrievalQA.from_chain_type(llm=llm, retriever=faiss_index.as_retriever(search_type="similarity", k=4))

def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        st.info("Listening... Speak now.")
        audio = recognizer.listen(source, phrase_time_limit=5)
        try:
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            st.warning("Sorry, could not understand the audio.")
        except sr.RequestError:
            st.error("Speech recognition service is unavailable.")
    return ""

def translate_to_arabic(text):
    try:
        # الترجمة من اللغة الأصلية إلى اللغة العربية
        return GoogleTranslator(source='auto', target='ar').translate(text)
    except Exception as e:
        return "تعذر الترجمة حالياً."

def generate_image_from_text(text):
    # This is a placeholder. Replace with real image generation if needed.
    img = Image.new('RGB', (300, 100), color=(73, 109, 137))
    return img

# Streamlit App
def main():
    st.set_page_config(page_title="Medical Chatbot (LLaMA 3 + RAG)", layout="centered")
    st.title("Medical Question Answering Chatbot (RAG + LLaMA 3)")

    if "qa_chain" not in st.session_state:
        st.session_state.qa_chain = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []
    if "selected_book" not in st.session_state:
        st.session_state.selected_book = list(PDF_PATHS.keys())[0]

    # Book selection buttons
    st.subheader("Select Book:")
    for book_name in PDF_PATHS:
        if st.button(book_name):
            st.session_state.selected_book = book_name
            st.session_state.qa_chain = None
            st.session_state.chat_history = []

    # Load model for selected book
    if st.session_state.qa_chain is None:
        with st.spinner(f"Loading {st.session_state.selected_book}..."):
            docs = load_medical_docs(PDF_PATHS[st.session_state.selected_book])
            faiss_index = embed_documents(docs)
            st.session_state.qa_chain = build_qa_system(faiss_index)

    # Input options
    col1, col2 = st.columns([3, 1])
    with col1:
        query = st.text_input("Ask a medical question:")
    with col2:
        if st.button("Use Voice"):
            voice_input = recognize_speech()
            if voice_input:
                st.success(f"You said: {voice_input}")
                query = voice_input

    # Answering
    if query:
        with st.spinner("Generating answer..."):
            result = st.session_state.qa_chain.run(query)
            translation = translate_to_arabic(result)
            st.session_state.chat_history.append({
                "question": query,
                "answer": result,
                "translated": translation
            })

    # Chat History
    st.markdown("---")
    st.subheader("Chat History")
    for chat in reversed(st.session_state.chat_history):
        st.markdown(f"**You:** {chat['question']}")
        st.markdown(f"**Bot:** {chat['answer']}")
        st.markdown(f"**ترجمة:** {chat['translated']}")

        if "image" in chat['answer'].lower() or "diagram" in chat['answer'].lower():
            if st.button(f"Generate Image for: {chat['question']}"):
                img = generate_image_from_text(chat['answer'])
                st.image(img, caption="Generated Illustration")

if __name__ == "__main__":
    main()


2025-05-09 20:29:00.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:29:00.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:29:00.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:29:00.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:29:00.780 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:29:00.781 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:29:00.781 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:29:00.782 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

1841


<ipython-input-11-b251b4b93c53>:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-05-09 20:33:13.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:33:13.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:33:13.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:33:13.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:33:13.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:33:13.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:33:13.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 20:33:13.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
from pyngrok import ngrok
!ngrok config add-authtoken 2w9WWgvNAVvctBnPUecvER45uQ0_2KgsjJkoPHM939isd4hrZ
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

!streamlit run medical_rag_chatbot.py &/content/Logs.txt &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://29b6-34-125-177-178.ngrok-free.app" -> "http://localhost:8501"
/bin/bash: line 1: /content/Logs.txt: No such file or directory
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: medical_rag_chatbot.py
